# Eli Lilly - List
## Wichtig
The following message was for 2017. Leave it here, never know, when it appears again.

Oh my friend, I am sorry... this PDF is strange. Special Characters (äöu) will not be detected. Tried id on many different ways. Work around:
* Open PDF in Adobe Pro
* Go to `PDF bearbeiten` (edit PDF)
* Select all
* Change Font to `Helvetiva Neue Light``
* Repeat for every page... :-(

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import pikepdf

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
df_list = tabula.read_pdf("EFPIA_2018_Switzerland.pdf", pages='all', lattice=True, multiple_tables=True)
df = pd.concat(df_list)

## Format PDF

In [7]:
df_export = df.copy()
#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total', 'empty2', 'empty3']

#Shift rows
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Add Type
df_export = df_export.reset_index(drop=True)
index_hco = df_export[df_export['name'].str.contains("Aggregate amount attributable", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove \r
f_export = remove_carination(df_export, " ")

#Remove Titles and Comma
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()
df_export['name'] = df_export.name.str.replace(r',$', '', regex=True, case=False)
df_export['name'] = df_export['name'].str.replace(", No Title", "")
df_export['name'] = df_export['name'].str.replace(",  Arzt", "")

#Remove empty
df_export.drop(columns=['empty1', 'empty2', 'empty3'], inplace=True)

#Convert to Numbers
df_export = df_export.replace('N/A', "")
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Add Fields
df_export = add_plz(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'eli')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)